In [2]:
!pip install colorspacious


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import mne
import pandas as pd
import os

from sklearn.decomposition import PCA
import scipy
from mne.decoding import CSP

from mpl_toolkits.axes_grid1 import make_axes_locatable, ImageGrid
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict
from mne.cov import _regularized_covariance, compute_whitener

In [6]:
%matplotlib auto
%matplotlib auto

Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg


In [7]:
def augmentation_ampl(dfs, vac):
    for i in dfs.keys():
        for ind in range(len(dfs[i])):
            peak = i.split("_")[1]
            name = str(int(dfs[i].iloc[ind][0]))
            if len(name) == 1:
                name = "0"+name
            print(name)
            amplitude_peak = dfs[i].iloc[ind][1]
            print(amplitude_peak)
            if name not in vac.keys():
                vac[name] = {}
                vac[name][peak] = amplitude_peak
            else:
                vac[name][peak] = amplitude_peak
            print(vac)
    return vac

In [9]:
def augmentation_time(dfs, vac):
    for i in dfs.keys():
        for ind in range(len(dfs[i])):
            peak = i.split("_")[1]
            name = str(int(dfs[i].iloc[ind][0]))
            if len(name) == 1:
                name = "0"+name
            time_peak = dfs[i].iloc[ind][1]
            if (int(time_peak*1000))%2 == 1:
                time_peak +=0.001
            if name not in vac.keys():
                vac[name] = {}
                vac[name][peak] = time_peak
            else:
                vac[name][peak] = time_peak
    return vac

In [10]:
a = pd.ExcelFile(r"C:\Users\Admin\Downloads\corrected_table.xlsx") # import table with latencys
sheet_names = a.sheet_names
latens_no = [name for name in sheet_names if "LATEN" in name and "NO" in name]
latens_yes = [name for name in sheet_names if "LATEN" in name and "YES" in name]
amplitudes_no = [name for name in sheet_names if "AMPL" in name and "NO" in name]
amplitudes_yes = [name for name in sheet_names if "AMPL" in name and "YES" in name]
dfs_no = {sheet: pd.read_excel(a, sheet, header = 1,names = None) for sheet in latens_no}
dfs_yes = {sheet: pd.read_excel(a, sheet, header = 1,names = None) for sheet in latens_yes}
dfs_ampl_no = {sheet: pd.read_excel(a, sheet, header = 1,names = None) for sheet in amplitudes_no}
dfs_ampl_yes = {sheet: pd.read_excel(a, sheet, header = 1,names = None) for sheet in amplitudes_yes}
t_no = {}
t_yes = {}
ampl_no = {}
ampl_yes = {}
t_no = augmentation_time(dfs_no, t_no)
t_yes = augmentation_time(dfs_yes, t_yes)
ampl_no = augmentation_ampl(dfs_ampl_no, ampl_no)
ampl_yes = augmentation_ampl(dfs_ampl_yes, ampl_yes)

04
6.8e-07
{'04': {'EARLY': 6.8e-07}}
05
1.3e-06
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}}
06
2.05e-06
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}, '06': {'EARLY': 2.05e-06}}
07
6e-08
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}, '06': {'EARLY': 2.05e-06}, '07': {'EARLY': 6e-08}}
10
6.1e-07
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}, '06': {'EARLY': 2.05e-06}, '07': {'EARLY': 6e-08}, '10': {'EARLY': 6.1e-07}}
11
5.9e-07
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}, '06': {'EARLY': 2.05e-06}, '07': {'EARLY': 6e-08}, '10': {'EARLY': 6.1e-07}, '11': {'EARLY': 5.9e-07}}
13
3.6e-07
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}, '06': {'EARLY': 2.05e-06}, '07': {'EARLY': 6e-08}, '10': {'EARLY': 6.1e-07}, '11': {'EARLY': 5.9e-07}, '13': {'EARLY': 3.6e-07}}
14
5.8e-07
{'04': {'EARLY': 6.8e-07}, '05': {'EARLY': 1.3e-06}, '06': {'EARLY': 2.05e-06}, '07': {'EARLY': 6e-08}, '10': {'EARLY': 6.1e-07}, '11': {'EARLY': 5.9e-07}, '13': {'EARLY': 3.6e-07},

In [11]:
def peaks_4_sex(evoked, ch, times):
    evoked_new = evoked.copy()
    evoked_4_ch = evoked.get_data()[evoked.ch_names.index(ch)]
    evoked_crop = evoked_4_ch[times-20: times+20]
    latency = np.argmax(evoked_crop)+times-20
    amplitude = np.max(evoked_crop)
    return latency, amplitude

In [12]:
def peaks_4_no_sex(evoked, ch, times):
    evoked_new = evoked.copy()
    evoked_4_ch = evoked_new.get_data()[epoch.ch_names.index(ch)]
    evoked_crop = evoked_4_ch[times-20: times+20]
    latency = np.argmin(evoked_crop)+times-20
    amplitude = np.min(evoked_crop)
    return latency, amplitude

In [13]:
def time (times,win):
    if win*1000%2 == 1:
        win+=0.001
    win = round(win, ndigits=4)
    print(win)
    time = np.where(times==win)
    print(time)
    times_new = time[0][0] 
    return times_new

In [19]:
data_dict = {}
timing = {}
conditions = ["yes", "no"]
roi_not_to_pick = ["Fp1", "Fp2", ""]
for a,b,c in os.walk(r"C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation"):
    for path in c:
        sname = path.split("_")[-1].split(".")[-2]
        path_2_epoch = a+"\\"+path
        epoch = mne.read_epochs(path_2_epoch, preload = True)
        evoked = {c:epoch[c].average() for c in conditions}
        t = evoked["yes"].times
        peaks_times_no = t_no[sname]
        peaks_times_yes = t_yes[sname]
        peaks_ampls_no = t_no[sname]
        peaks_ampls_yes = t_yes[sname]
        if sname not in timing.keys():
            timing[sname] = {"yes":{}, "no":{}}
            data_dict[sname] = {"yes":{}, "no":{}}
        for peak in peaks_ampls_no.keys():
            times_no = time(t, peaks_times_no[peak])
            times_yes = time(t, peaks_times_yes[peak])
            timing[sname]["yes"][peak] = times_yes
            timing[sname]["no"][peak] = times_no
        for cond in evoked.keys():
            evoked_chs = evoked[cond]
            print(cond)
            if cond == "no":
                print(evoked_chs)
                for ch in evoked_chs.ch_names:
                    data_dict[sname][cond][ch] = {}
                    times_early, amplitudes_early = peaks_4_sex(evoked_chs, ch, timing[sname][cond]["EARLY"])
                    times_negative, amplitudes_negative = peaks_4_no_sex(evoked_chs, ch,
                                                                         timing[sname][cond]["NEGATIVE"])
                    times_late, amplitudes_late = peaks_4_sex(evoked_chs, ch, timing[sname][cond]["LATE"])
                    data_dict[sname][cond][ch] = {"EARLY AMPL":amplitudes_early, "EARLY LATEN":times_early,
                                             "NEGATIVE AMPL":amplitudes_negative, "NEGATIVE LATEN":times_negative,
                                             "LATE AMPL":amplitudes_late, "LATE LATEN":times_late}
            else:
                for ch in evoked_chs.ch_names:
                    data_dict[sname][cond][ch] = {}
                    times_early, amplitudes_early = peaks_4_sex(evoked_chs, ch, timing[sname][cond]["EARLY"])
                    times_negative, amplitudes_negative = peaks_4_sex(evoked_chs, ch,
                                                                      timing[sname][cond]["NEGATIVE"])
                    times_late, amplitudes_late = peaks_4_sex(evoked_chs, ch, timing[sname][cond]["LATE"])
                    data_dict[sname][cond][ch] = {"EARLY AMPL":amplitudes_early, "EARLY LATEN":times_early,
                                              "NEGATIVE AMPL":amplitudes_negative, "NEGATIVE LATEN":times_negative,
                                              "LATE AMPL":amplitudes_late, "LATE LATEN":times_late}   

Reading C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_04.fif ...
    Found the data of interest:
        t =   -1000.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
178 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\Admin\AppData\Local\Temp\ipykernel_14256\1398101531.py:9: RuntimeWarning: This filename (C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_04.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoch = mne.read_epochs(path_2_epoch, preload = True)


0.176
(array([588], dtype=int64),)
0.166
(array([583], dtype=int64),)
0.308
(array([654], dtype=int64),)
0.222
(array([611], dtype=int64),)
0.462
(array([731], dtype=int64),)
0.36
(array([680], dtype=int64),)
yes
no
<Evoked | 'no' (average, N=97), -1 – 3 s, baseline -1 – 0 s, 31 ch, ~534 kB>
Reading C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_05.fif ...
    Found the data of interest:
        t =   -1000.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
162 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
0.332
(array([666], dtype=int64),)
0.286
(array([643], dtype=int64),)
0.538
(array([769], dtype=int64),)
0.46
(array([730], dtype=int64),)
0.472
(array([736], dtype=int64),)
0.568
(array([784], dtype=int64),)
yes
no
<Evoked | 'no' (averag

C:\Users\Admin\AppData\Local\Temp\ipykernel_14256\1398101531.py:9: RuntimeWarning: This filename (C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_05.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoch = mne.read_epochs(path_2_epoch, preload = True)


Reading C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_06.fif ...
    Found the data of interest:
        t =   -1000.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
165 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
0.186
(array([593], dtype=int64),)
0.184
(array([592], dtype=int64),)
0.398
(array([699], dtype=int64),)
0.27
(array([635], dtype=int64),)
0.568
(array([784], dtype=int64),)
0.322
(array([661], dtype=int64),)
yes
no
<Evoked | 'no' (average, N=101), -1 – 3 s, baseline -1 – 0 s, 31 ch, ~534 kB>
Reading C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_18.fif ...
    Found the data of interest:
        t =   -1000.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata


C:\Users\Admin\AppData\Local\Temp\ipykernel_14256\1398101531.py:9: RuntimeWarning: This filename (C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_18.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoch = mne.read_epochs(path_2_epoch, preload = True)


194 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
0.19
(array([595], dtype=int64),)
0.246
(array([623], dtype=int64),)
0.298
(array([649], dtype=int64),)
0.32
(array([660], dtype=int64),)
0.37
(array([685], dtype=int64),)
0.382
(array([691], dtype=int64),)
yes
no
<Evoked | 'no' (average, N=97), -1 – 3 s, baseline -1 – 0 s, 31 ch, ~534 kB>
Reading C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_19.fif ...
    Found the data of interest:
        t =   -1000.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata


C:\Users\Admin\AppData\Local\Temp\ipykernel_14256\1398101531.py:9: RuntimeWarning: This filename (C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_19.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoch = mne.read_epochs(path_2_epoch, preload = True)


199 matching events found
No baseline correction applied
0 projection items activated
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
0.168
(array([584], dtype=int64),)
0.168
(array([584], dtype=int64),)
0.24
(array([620], dtype=int64),)
0.27
(array([635], dtype=int64),)
0.356
(array([678], dtype=int64),)
0.344
(array([672], dtype=int64),)
yes
no
<Evoked | 'no' (average, N=100), -1 – 3 s, baseline -1 – 0 s, 31 ch, ~534 kB>


In [20]:
ref_fz_dict = {}
for name in data_dict.keys():
    if name not in ref_fz_dict.keys():
        ref_fz_dict[name] = {}
    for condition in data_dict[sname].keys():
        if condition not in ref_fz_dict[name].keys():
            ref_fz_dict[name][condition] = {}
        ref_ch = data_dict[sname][condition]["Fz"]
        time_average_ref = ref_ch["EARLY LATEN"]+ref_ch["NEGATIVE LATEN"]+ref_ch["LATE LATEN"]/3
        print("______________")
        print(time_average_ref)
        ampl_early_ref = ref_ch["EARLY AMPL"]
        ampl_neg_ref = ref_ch["NEGATIVE AMPL"]
        ampl_late_ref = ref_ch['LATE AMPL']
        for ch in data_dict[name][condition].keys():
            data_ch = data_dict[sname][condition][ch]
            time_average = data_ch["EARLY LATEN"]+data_ch["NEGATIVE LATEN"]+data_ch["LATE LATEN"]/3
            time_diff = int(time_average_ref - time_average)
            ampl_early_ch = data_ch["EARLY AMPL"]
            ampl_neg_ch = data_ch["NEGATIVE AMPL"]
            ampl_late_ch = data_ch["LATE AMPL"]
            ampl_early_diff = ampl_early_ch/ampl_early_ref
            ampl_neg_diff = ampl_neg_ch/ampl_neg_ref
            ampl_late_diff = ampl_late_ch/ampl_late_ref
            print(time_average)
            print("--------")
            ref_fz_dict[name][condition][ch] = {"TIME DIFF":time_diff,"TIME EARLY":round(data_ch["EARLY LATEN"], ndigits = 0),
                                                "TIME NEGATIVE":round(data_ch["NEGATIVE LATEN"]),
                                                "TIME LATE":round(data_ch["LATE LATEN"], ndigits = 0), "AMPL EARLY":ampl_early_diff, 
                                               "AMPL NEGATIVE": ampl_neg_diff, "AMPL LATE":ampl_late_diff}

______________
1458.3333333333333
1458.6666666666667
--------
1458.3333333333333
--------
1458.3333333333333
--------
1459.6666666666667
--------
1460.3333333333333
--------
1459.3333333333333
--------
1459.6666666666667
--------
1469.3333333333333
--------
1458.6666666666667
--------
1429.0
--------
1473.6666666666667
--------
1465.0
--------
1468.0
--------
1477.3333333333333
--------
1437.3333333333333
--------
1438.0
--------
1438.3333333333333
--------
1438.0
--------
1477.6666666666667
--------
1396.3333333333333
--------
1469.6666666666667
--------
1469.3333333333333
--------
1458.6666666666667
--------
1469.3333333333333
--------
1457.0
--------
1450.3333333333333
--------
1456.6666666666667
--------
1456.3333333333333
--------
1456.3333333333333
--------
1459.0
--------
1459.6666666666667
--------
______________
1440.0
1435.0
--------
1440.0
--------
1442.0
--------
1435.6666666666667
--------
1434.6666666666667
--------
1444.0
--------
1446.0
--------
1446.0
--------
1448.666

In [21]:
ref_fz_dict

{'04': {'yes': {'Fp1': {'TIME DIFF': 0,
    'TIME EARLY': 581,
    'TIME NEGATIVE': 654,
    'TIME LATE': 671,
    'AMPL EARLY': 1.2112890946671726,
    'AMPL NEGATIVE': -0.5659335720306742,
    'AMPL LATE': 0.2857135801396063},
   'Fz': {'TIME DIFF': 0,
    'TIME EARLY': 581,
    'TIME NEGATIVE': 654,
    'TIME LATE': 670,
    'AMPL EARLY': 1.0,
    'AMPL NEGATIVE': 1.0,
    'AMPL LATE': 1.0},
   'F3': {'TIME DIFF': 0,
    'TIME EARLY': 581,
    'TIME NEGATIVE': 654,
    'TIME LATE': 670,
    'AMPL EARLY': 1.0780614136737232,
    'AMPL NEGATIVE': 0.6811118564033833,
    'AMPL LATE': 0.732828602707455},
   'F7': {'TIME DIFF': -1,
    'TIME EARLY': 582,
    'TIME NEGATIVE': 654,
    'TIME LATE': 671,
    'AMPL EARLY': 1.1631164494113158,
    'AMPL NEGATIVE': -0.5963521790227714,
    'AMPL LATE': 0.06545954754579633},
   'FT9': {'TIME DIFF': -2,
    'TIME EARLY': 582,
    'TIME NEGATIVE': 654,
    'TIME LATE': 673,
    'AMPL EARLY': 0.8997057667928838,
    'AMPL NEGATIVE': -0.94177623488

In [25]:
epochs = mne.read_epochs(r"C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_04.fif", 
                         preload = True)
name = "04"
epochs_new = []
for num in range(len(epochs)):
    epo = epochs[num].get_data().copy()
    cond = list(epochs[num].event_id)[0]
    epo_ref = epo[0][epochs.ch_names.index("Fz")]
    epoch_after_aug = []
#     print(epo_ref.shape)
    for chan in range(len(epochs.ch_names)):
        ch = epochs.ch_names[chan]
        ch_2_check = epochs.ch_names.index(ch)
        if ch_2_check == chan:
            print("COOOOOOOOOOL")
        else:
            raise "ERRROR DOLBOEB"
        info = ref_fz_dict[name][cond][ch]
        time_diff = ref_fz_dict[name][cond][ch]["TIME DIFF"]
        time_early = ref_fz_dict[name][cond][ch]["TIME EARLY"]
        time_neg = ref_fz_dict[name][cond][ch]["TIME NEGATIVE"]
        time_late = ref_fz_dict[name][cond][ch]["TIME LATE"]
        if time_diff>0:
            epo_4_concat = epo_ref[-time_diff-1:0]
        elif time_diff<0:
            epo_4_concat = epo_ref[0:time_diff]
        elif time_diff == 0:
            epo_4_concat = []
        length = 0
        if int(time_diff)<0:
            TAIL_BEFORE = epo_ref[:500]
#             print(len(TAIL_BEFORE))
            ERP_EARLY = epo_ref[500:time_early+10]
#             print(len(ERP_EARLY), time_early)
            ERP_NEGATIVE = epo_ref[time_early+10:time_neg+20]
#             print(len(ERP_NEGATIVE), time_neg)
            ERP_LATE = epo_ref[time_neg+20:time_late+40]
#             print(len(ERP_LATE), time_late)
            TAIL_AFTER = epo_ref[time_late+40:]
#             print(len(TAIL_AFTER))
#             print(len(TAIL_AFTER))            
        elif int(time_diff)>0:
            TAIL_BEFORE = epo_ref[:500]
#             print(len(TAIL_BEFORE))
            ERP_EARLY = epo_ref[500:time_early+10]
#             print(len(ERP_EARLY), time_early)
            ERP_NEGATIVE = epo_ref[time_early+10:time_neg+20]
#             print(len(ERP_NEGATIVE), time_neg)
            ERP_LATE = epo_ref[time_neg+20:time_late+40]
#             print(len(ERP_LATE), time_late)
            TAIL_AFTER = epo_ref[time_late+40:]
#             print(len(TAIL_AFTER))
#             print(len(TAIL_AFTER))            
        else:
            TAIL_BEFORE = epo_ref[:500]
#             print(len(TAIL_BEFORE))
            ERP_EARLY = epo_ref[500:time_early+10]
#             print(len(ERP_EARLY), time_early)
            ERP_NEGATIVE = epo_ref[time_early+10:time_neg+20]
#             print(len(ERP_NEGATIVE), time_neg)
            ERP_LATE = epo_ref[time_neg+20:time_late+40]
#             print(len(ERP_LATE), time_late)
            TAIL_AFTER = epo_ref[time_late+40:]
#             print(len(TAIL_AFTER))
#             print(len(TAIL_AFTER))            
#         print("____________________")
#         print(len(ERP_LATE)+len(ERP_NEGATIVE)+len(ERP_EARLY)+len(TAIL_BEFORE)+len(TAIL_AFTER))
#         print("====================")
        ERP_EARLY = ERP_EARLY*info["AMPL EARLY"]
        ERP_NEGATIVE = ERP_NEGATIVE*info["AMPL NEGATIVE"]
        ERP_LATE = ERP_LATE*info["AMPL LATE"]
        mean = np.median([info["AMPL EARLY"], info["AMPL LATE"], info["AMPL NEGATIVE"]])
        TAIL_AFTER = TAIL_AFTER*mean
        TAIL_BEFORE = TAIL_BEFORE*mean
        if time_diff>0:
            new_epo = np.concatenate([TAIL_BEFORE, ERP_EARLY, ERP_NEGATIVE, ERP_LATE, TAIL_AFTER])
        elif time_diff<0:
            new_epo = np.concatenate([TAIL_BEFORE, ERP_EARLY, ERP_NEGATIVE, ERP_LATE, TAIL_AFTER])
        else:
            new_epo = np.concatenate([TAIL_BEFORE, ERP_EARLY, ERP_NEGATIVE, ERP_LATE, TAIL_AFTER])
        print(len(new_epo))
        if len(new_epo) !=2001:
            raise ERROR
        epoch_after_aug.append(new_epo)
    epochs_new.append(epoch_after_aug)
    plt.figure()
    plt.plot([epochs[num].times][0], epo_ref)
    plt.plot([epochs[num].times][0], new_epo)
    plt.title(f"{ch} vs Fz")

Reading C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_04.fif ...
    Found the data of interest:
        t =   -1000.00 ...    3000.00 ms
        0 CTF compensation matrices available
Not setting metadata
178 matching events found
No baseline correction applied
0 projection items activated


C:\Users\Admin\AppData\Local\Temp\ipykernel_14256\999873655.py:1: RuntimeWarning: This filename (C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_04.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(r"C:\Users\Admin\Desktop\augmentations\epochs_4_augmentation\subj_04.fif",


COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO

COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO

C:\Users\Admin\AppData\Local\Temp\ipykernel_14256\999873655.py:86: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


...

COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO

COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO

COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO

COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO

COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOOOL
2001
COOOOOOOOO